In [55]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from main import load_and_prepare_sessions
from processing.session_sampling import MiceAnalysis
import numpy as np

sessions = load_and_prepare_sessions("../../Baseline", load_from_pickle=True, remove_bad_signal_sessions=True)
mouse_analyser = MiceAnalysis(sessions)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# the new performance metrics are avg time from disp to hit, avg time from hit to reward and
# ITI touches

In [57]:
def trim_to_match_sequences(dispimg_idxs, hit_idxs, reward_idxs):
    
    # Step 1: Ensure each 'dispimg' has a 'hit' after it
    hit_after_dispimg_mask = np.searchsorted(hit_idxs, dispimg_idxs, side='right') < len(hit_idxs)
    dispimg_idxs = dispimg_idxs[hit_after_dispimg_mask]
    
    # Step 2: Ensure each 'hit' has a 'reward' after it
    # This also indirectly ensures that each 'dispimg' has a corresponding 'reward' due to the previous step
    reward_after_hit_mask = np.searchsorted(reward_idxs, hit_idxs, side='right') < len(reward_idxs)
    hit_idxs = hit_idxs[reward_after_hit_mask]
    
    # Final Alignment: In case lengths are mismatched after trimming, align to the shortest sequence
    min_length = min(len(dispimg_idxs), len(hit_idxs), len(reward_idxs))
    dispimg_idxs = dispimg_idxs[:min_length]
    hit_idxs = hit_idxs[:min_length]
    reward_idxs = reward_idxs[np.searchsorted(reward_idxs, hit_idxs[:min_length], side='right') - 1]  # Matching rewards to hits

    return dispimg_idxs, hit_idxs, reward_idxs


def get_avg_time_from_disp_to_hit_to_reward(sessions):
    dispimg_idxs = []
    hit_idxs = []
    reward_idxs = []
    
    for session in sessions:
        event_idxs_data = session.event_idxs_container.data
        phot_df = session.df_container.get_data("photwrit_470")
        phot_times = phot_df['SecFromZero_FP3002'].values
        
        curr_dispimg_idxs = event_idxs_data['before_dispimg_hit']
        curr_hit_idxs = event_idxs_data['hit']
        curr_reward_idxs = event_idxs_data['reward_collect']

        min_len = min(len(curr_dispimg_idxs), len(curr_hit_idxs), len(curr_reward_idxs))

        curr_dispimg_idxs = curr_dispimg_idxs[:min_len]
        curr_hit_idxs = curr_hit_idxs[:min_len]
        curr_reward_idxs = curr_reward_idxs[:min_len]

        dispimg_idxs.extend(curr_dispimg_idxs)
        hit_idxs.extend(curr_hit_idxs)
        reward_idxs.extend(curr_reward_idxs)

    dispimg_idxs = np.array(dispimg_idxs)
    hit_idxs = np.array(hit_idxs)
    reward_idxs = np.array(reward_idxs)

    # Perform element-wise comparison
    if not ((dispimg_idxs < hit_idxs) & (hit_idxs < reward_idxs)).all():
        raise ValueError("Indices are not in order")

    return (np.mean(phot_times[hit_idxs] - phot_times[dispimg_idxs]),
            np.mean(phot_times[reward_idxs] - phot_times[hit_idxs])) 

In [64]:
all_items = sessions[0].df_container.get_data("raw")['Item_Name'].count("Centre_Touches_during_ITI")


{'(SYSTEM)',
 'A1Sound1',
 'A2Sound2',
 'A3Sound3',
 'Adjust Sound Level 2',
 'After_Reward_Pause',
 'After_Reward_Timer',
 'Background',
 'Blank Image Touched',
 'Blank_Touch_Counter',
 'Bussey Mouse Operant Mode 3 x 1 x low',
 'Centre Screen Touches',
 'Centre_Touches_during_ITI',
 'Corr_Trial_Image_Selection',
 'Correct Rejection',
 'Correct_Grid_Position',
 'Correct_Image',
 'Correction Trial Correct Rejection',
 'Correction_Trial',
 'Correction_Trial_Correct_Rejections',
 'Correction_Trial_Mistakes',
 'Current_ITI',
 'Current_Image',
 'Display Image',
 'FIRBeam #1',
 'Feeder #1',
 'Feeder_Pulse_Time',
 'First_Trial',
 'Frame',
 'Free starting pellet',
 'Group Change',
 'Hit',
 'Hit_TTL #1',
 'Houselight_Normally_On',
 'ITI_Timer',
 'ITIs',
 'Image_Selection',
 'Images',
 'Increment Trial',
 'Left_Touches_during_ITI',
 'Limited_Hold',
 'Limited_Hold_Timer',
 'Max_no_of_rewards',
 'Missed Hit',
 'Mistake',
 'No_of_Misses',
 'No_of_Mistakes',
 'No_of_correct_rejections',
 'No_of_hits